First assemble a whole big dataset named `small_passenge.json`, using the [Nvidia dataset](https://huggingface.co/datasets/nvidia/ChatQA-Training-Data). That datasets include files like drop/train.json, narrativeqa/train.json, etc, which will be used below. Modify their file paths to your need.



In [ ]:
import json
import pandas as pd

empty_df = pd.DataFrame()
files=["ChatQA-Training-Data/drop/train.json",
       "ChatQA-Training-Data/narrativeqa/train.json",
       "ChatQA-Training-Data/quoref/train.json",
       "ChatQA-Training-Data/newsqa/train.json",
       "ChatQA-Training-Data/ropes/train.json",
       "ChatQA-Training-Data/squad1.1/train.json",
       "ChatQA-Training-Data/squad2.0/train.json",
       "ChatQA-Training-Datat/synthetic_convqa/train.json",
       "ChatQA-Training-Data/tatqa/train_arithmetic.json",
       "ChatQA-Training-Data/tatqa/train_others.json"]
for file in files:
    data = pd.read_json(file)
    data_unique = data.drop_duplicates(subset=['document'])
    data_unique=data_unique[['document']]
    empty_df= pd.concat([empty_df, data_unique])
empty_df = empty_df.drop_duplicates(subset=['document'])

empty_df.to_json('small_passenge.json', orient='records', indent=4, date_format='iso')


Now generate our chunking dataset `mixed_passenge_train_30000.json`. We sample from small_passenge.json 4 different rows to cross them into one row of chunking dataset `mixed_passenge_train_30000.json`, which they are separated by symbol "<--start-->". We generate 30000 of rows for chunking dataset. 

In [ ]:
START_SYM="<--start-->"
file="/data/full_data.jsonl"
import pandas as pd
data = pd.read_json(file,lines=True)
empty_df = pd.DataFrame()
background_sentence_num=4
avrge_tokens_per_sentence=20
max_tokens=256

import random
# import nltk
from tqdm import tqdm
# nltk.download('punkt')

# from nltk.tokenize import sent_tokenize

for i in tqdm(range(0, 15000)):
    sam = data.sample(n=1)
    res = ""

    for index, row in sam.iterrows():
        allsen = row["story_zh"].split('。')
        allsen = [sents+'。' for sents in allsen if len(sents.strip())>0]
        ranges = random.sample(range(len(allsen)+1), 2)
        ranges.sort()
        ss = allsen[ranges[0]:ranges[1]]
        res = "".join(ss)
        break
    sam = pd.DataFrame({"input": [res]})
    empty_df = pd.concat([empty_df, sam])

for i in tqdm(range(0, 15000)):
    sam = data.sample(n=2)
    res = ""

    for index, row in sam.iterrows():

        sentence_num = random.randint(1, 5)
        allsen = row["story_zh"].split('。')
        allsen = [sents+'。' for sents in allsen if len(sents.strip())>0]
        ranges = random.sample(range(len(allsen)+1), 2)
        ranges.sort()
        ss = allsen[ranges[0]:ranges[1]]
        ss = "".join(ss)
        res = res + START_SYM + ss

    res = res[11:]
    sam = pd.DataFrame({"input": [res]})
    empty_df = pd.concat([empty_df, sam])

for i in tqdm(range(0, 15000)):
    sam = data.sample(n=3)
    res = ""

    for index, row in sam.iterrows():

        sentence_num = random.randint(1, 5)
        allsen = row["story_zh"].split('。')
        allsen = [sents+'。' for sents in allsen if len(sents.strip())>0]
        ranges = random.sample(range(len(allsen)+1), 2)
        ranges.sort()
        ss = allsen[ranges[0]:ranges[1]]
        ss = "".join(ss)
        res = res + START_SYM + ss

    res = res[11:]
    sam = pd.DataFrame({"input": [res]})
    empty_df = pd.concat([empty_df, sam])
    
for i in tqdm(range(0, 15000)):
    sam = data.sample(n=4)
    res = ""

    for index, row in sam.iterrows():

        sentence_num = random.randint(1, 5)
        allsen = row["story_zh"].split('。')
        allsen = [sents+'。' for sents in allsen if len(sents.strip())>0]
        ranges = random.sample(range(len(allsen)+1), 2)
        ranges.sort()
        ss = allsen[ranges[0]:ranges[1]]
        ss = "".join(ss)
        res = res + START_SYM + ss

    res = res[11:]
    sam = pd.DataFrame({"input": [res]})
    empty_df = pd.concat([empty_df, sam])
    
empty_df = empty_df.sample(frac=1).reset_index(drop=True)
empty_df = empty_df.drop_duplicates(subset=["input"])
empty_df.to_json(
    "mixed_passenge_train.json",
    orient="records",
    indent=4,
    date_format="iso",
)

Run the same code but to generate `mixed_passenge_eval_30000.json` for evaluation.

In [24]:
import json
with open('/data/BertChunker-zh/mixed_passenge_train.json','r',encoding='utf-8') as f:
    mixed_passenge_train_40000 = json.loads(f.read())
mixed_passenge_train_30000 = mixed_passenge_train_40000[:50000]
mixed_passenge_test_10000 = mixed_passenge_train_40000[50000:]
with open('/data/BertChunker-zh/mixed_passenge_train.json','w',encoding='utf-8') as f:
    f.write(json.dumps(mixed_passenge_train_30000))
with open('/data/BertChunker-zh/mixed_passenge_test.json','w',encoding='utf-8') as f:
    f.write(json.dumps(mixed_passenge_test_10000))

In [ ]:
with open('/data/BertChunker-zh/mixed_passenge_test.json','r',encoding='utf-8') as f:
    data=json.loads(f.read())
print(data[100]['input'])